### Checking Python Environment Path

In [46]:
import sys
print(sys.executable)

/opt/conda/bin/python3.10


### Importing Libraries

In [47]:
import csv
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence  import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

# Data Analyzing and Cleaning

### Importing CSV files and merging them all together

In [48]:
data = pd.concat(
    map(pd.read_csv, [
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor1.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor2.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor3.csv',
                      '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor4.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor5.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor6.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2018Floor7.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor1.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor2.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor3.csv',
                      '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor4.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor5.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor6.csv',
#                       '/kaggle/input/cubems-smart-building-energy-and-iaq-data/2019Floor7.csv',
                      
                      
                     ]),
    ignore_index=True)

data.head(5)

,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
0,2018-07-01 00:00:00,0.0,0.0,0.0,8.74,0.00,0.22,28.28,62.50,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.15,27.73,62.86,0.0
1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.73,62.87,0.0
2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790560 entries, 0 to 790559
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          790560 non-null  object 
 1   z1_AC1(kW)    621829 non-null  float64
 2   z1_AC2(kW)    621829 non-null  float64
 3   z1_AC3(kW)    621829 non-null  float64
 4   z1_AC4(kW)    751579 non-null  float64
 5   z1_Light(kW)  707605 non-null  float64
 6   z1_Plug(kW)   756532 non-null  float64
 7   z1_S1(degC)   526125 non-null  float64
 8   z1_S1(RH%)    526128 non-null  float64
 9   z1_S1(lux)    526127 non-null  float64
 10  z2_AC1(kW)    751576 non-null  float64
 11  z2_Light(kW)  707605 non-null  float64
 12  z2_Plug(kW)   707605 non-null  float64
 13  z2_S1(degC)   519026 non-null  float64
 14  z2_S1(RH%)    519026 non-null  float64
 15  z2_S1(lux)    519027 non-null  float64
 16  z3_Light(kW)  756457 non-null  float64
 17  z3_Plug(kW)   756490 non-null  float64
 18  z4_A

### Sorting Dataframe values in order of Date & Time

In [50]:
# df['Date'] = pd.to_datetime(df['Date'])
data = data.sort_values(by='Date')

In [51]:
data.head(5)

,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
0,2018-07-01 00:00:00,0.0,0.0,0.0,8.74,0.00,0.22,28.28,62.50,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.15,27.73,62.86,0.0
1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.73,62.87,0.0
2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0


In [52]:
data = data[1:]
data.head(5)

,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.73,62.87,0.0
2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
5,2018-07-01 00:05:00,0.0,0.0,0.0,8.68,0.00,0.23,28.29,62.51,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0


### Converting 'Date' column into DateTime Format

In [53]:
data['Date'] = pd.to_datetime(data['Date'], infer_datetime_format = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 790559 entries, 1 to 790559
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          790559 non-null  datetime64[ns]
 1   z1_AC1(kW)    621828 non-null  float64       
 2   z1_AC2(kW)    621828 non-null  float64       
 3   z1_AC3(kW)    621828 non-null  float64       
 4   z1_AC4(kW)    751578 non-null  float64       
 5   z1_Light(kW)  707604 non-null  float64       
 6   z1_Plug(kW)   756531 non-null  float64       
 7   z1_S1(degC)   526124 non-null  float64       
 8   z1_S1(RH%)    526127 non-null  float64       
 9   z1_S1(lux)    526126 non-null  float64       
 10  z2_AC1(kW)    751575 non-null  float64       
 11  z2_Light(kW)  707604 non-null  float64       
 12  z2_Plug(kW)   707604 non-null  float64       
 13  z2_S1(degC)   519025 non-null  float64       
 14  z2_S1(RH%)    519025 non-null  float64       
 15  z2_S1(lux)    519

### Graphs for different Devices and Zones

In [54]:

# data.set_index('Date')[['z1_Light(kW)', 'z1_Plug(kW)', 'z2_AC1(kW)', 'z2_AC2(kW)', 'z2_AC3(kW)', 'z2_AC4(kW)', 'z2_Light(kW)',
#            'z2_Plug(kW)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_Light(kW)', 'z1_AC1(kW)', 'z2_AC5(kW)', 'z2_AC6(kW)',
#            'z2_AC7(kW)', 'z2_AC8(kW)', 'z2_AC9(kW)', 'z2_AC10(kW)', 'z2_AC11(kW)', 'z2_AC12(kW)', 'z2_AC13(kW)',
#            'z2_AC14(kW)', 'z4_AC1(kW)', 'z4_Plug(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)', 'z5_AC1(kW)','z5_Light(kW)',
#             'z5_Plug(kW)', 'z4_AC2(kW)', 'z4_AC3(kW)', 'z4_AC4(kW)', 'z1_S1(degC)', 'z2_S1(degC)', 'z3_S1(degC)', 'z4_S1(degC)',
#             'z5_S1(degC)', 'z1_S1(RH%)', 'z2_S1(RH%)', 'z3_S1(RH%)', 'z4_S1(RH%)', 'z5_S1(RH%)', 'z1_S1(lux)', 'z2_S1(lux)',
#             'z3_S1(lux)', 'z4_S1(lux)', 'z5_S1(lux)']].plot(subplots=True, figsize=(15,15))


# plt.figure(figsize=(14,5))
# sns.set_style("ticks")
# sns.lineplot(x=data['Date'],y=data['z1_Light(kW)'],color='blue')
# sns.despine()
# plt.title("Zone 1 Light Power Consumption(kW)",size='x-large',color='blue')

data.head()




,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.73,62.87,0.0
2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,0.0,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
5,2018-07-01 00:05:00,0.0,0.0,0.0,8.68,0.00,0.23,28.29,62.51,0.0,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0


### Missing Values Heatmap

In [55]:
# num_ticks=19 #tick number for Y-axis
# plt.figure(figsize = (10,5))
# ax = sns.heatmap(data.isnull(), cbar=False)
# ax.set_yticks(np.linspace(0,100,num_ticks,dtype=np.integer))
# ax.set_title('Missing Values')
# plt.show()

In [56]:
data.reset_index()

,index,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
0,1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.73,62.87,0.0
1,2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
2,3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
3,4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,...,0.07,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
4,5,2018-07-01 00:05:00,0.0,0.0,0.0,8.68,0.00,0.23,28.29,62.51,...,0.08,NaN,NaN,NaN,0.0,0.0,0.14,27.72,62.88,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790554,790555,2019-12-31 23:55:00,0.0,0.0,0.0,0.00,0.00,0.23,27.62,63.19,...,0.05,26.45,66.64,0.0,0.0,0.0,0.19,27.80,67.67,0.0
790555,790556,2019-12-31 23:56:00,0.0,0.0,0.0,0.00,0.01,0.23,27.63,63.17,...,0.03,26.45,66.68,0.0,0.0,0.0,0.14,27.80,67.70,0.0
790556,790557,2019-12-31 23:57:00,0.0,0.0,0.0,0.00,0.00,0.23,27.63,63.17,...,0.05,26.45,66.68,0.0,0.0,0.0,0.18,27.70,67.70,0.0
790557,790558,2019-12-31 23:58:00,0.0,0.0,0.0,0.00,0.00,0.23,27.62,63.18,...,0.03,26.45,66.65,0.0,0.0,0.0,0.14,27.80,67.70,0.0


### Solving Blank Cell Problem assuming Consumption Value '0'

In [57]:
data = data.fillna(0)

In [58]:
data.describe()

,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),z2_AC1(kW),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
count,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,...,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000,790559.000000
mean,0.000309,0.009051,0.000966,4.514454,1.417981,0.963996,17.086992,41.557118,15.470694,3.220222,...,0.389162,13.619430,35.080700,13.975827,1.656883,0.469562,0.224868,16.189863,44.723494,12.273500
std,0.030544,0.133802,0.041527,9.664746,1.881480,1.066666,12.297406,29.713895,25.936497,7.003122,...,0.451110,12.448004,31.773315,25.024414,3.898456,0.725200,0.214992,11.720073,32.352634,25.061398
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,...,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.380000,24.820000,59.310000,0.000000,0.000000,...,0.100000,18.980000,56.450000,0.000000,0.000000,0.000000,0.160000,23.580000,63.110000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,3.270000,1.400000,26.680000,63.790000,31.000000,0.000000,...,0.770000,25.340000,63.990000,8.000000,0.000000,1.010000,0.310000,25.190000,69.710000,1.000000
max,22.290000,20.280000,9.910000,51.690000,63.110000,87.110000,31.900000,78.800000,87.000000,58.400000,...,9.520000,30.990000,85.300000,111.000000,56.460000,58.610000,3.140000,30.900000,84.810000,87.000000


In [59]:
data.head(5)

,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),...,z4_Plug(kW),z4_S1(degC),z4_S1(RH%),z4_S1(lux),z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux)
1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,0.0,...,0.07,0.0,0.0,0.0,0.0,0.0,0.14,27.73,62.87,0.0
2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,0.0,...,0.08,0.0,0.0,0.0,0.0,0.0,0.14,27.72,62.88,0.0
3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,0.0,...,0.08,0.0,0.0,0.0,0.0,0.0,0.14,27.72,62.88,0.0
4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,0.0,...,0.07,0.0,0.0,0.0,0.0,0.0,0.14,27.72,62.88,0.0
5,2018-07-01 00:05:00,0.0,0.0,0.0,8.68,0.00,0.23,28.29,62.51,0.0,...,0.08,0.0,0.0,0.0,0.0,0.0,0.14,27.72,62.88,0.0


### Defining Target Variable

 Here We have converted Lux, DegC and Rh% into kW units for Data simplicity 
(due to Consideration of GPU power and time)

kW  = Lux/100

kW  = [Temperature(degC)*2] + [Humidity(Rh%)/2]

In [62]:
# load_kw = ['z1_Light(kW)', 'z1_Plug(kW)', 'z2_AC1(kW)', 'z2_AC2(kW)', 'z2_AC3(kW)', 'z2_AC4(kW)', 'z2_Light(kW)',
#            'z2_Plug(kW)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_Light(kW)', 'z1_AC1(kW)', 'z2_AC5(kW)', 'z2_AC6(kW)',
#            'z2_AC7(kW)', 'z2_AC8(kW)', 'z2_AC9(kW)', 'z2_AC10(kW)', 'z2_AC11(kW)', 'z2_AC12(kW)', 'z2_AC13(kW)',
#            'z2_AC14(kW)', 'z4_AC1(kW)', 'z4_Plug(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)', 'z5_AC1(kW)',
#            'z5_Light(kW)', 'z5_Plug(kW)', 'z4_AC2(kW)', 'z4_AC3(kW)', 'z4_AC4(kW)' ]

# load_degC = ['z1_S1(degC)', 'z2_S1(degC)', 'z3_S1(degC)', 'z4_S1(degC)', 'z5_S1(degC)']

# load_rh = ['z1_S1(RH%)', 'z2_S1(RH%)', 'z3_S1(RH%)', 'z4_S1(RH%)', 'z5_S1(RH%)' ]

# load_lux = ['z1_S1(lux)', 'z2_S1(lux)', 'z3_S1(lux)', 'z4_S1(lux)', 'z5_S1(lux)' ]

load_kw = ['z1_Light(kW)', 'z1_Plug(kW)', 'z2_AC1(kW)', 'z2_Light(kW)',
           'z2_Plug(kW)', 'z3_Light(kW)', 'z3_Plug(kW)', 'z4_Light(kW)', 'z4_AC1(kW)', 'z4_Plug(kW)', 'z1_AC2(kW)', 'z1_AC3(kW)', 'z1_AC4(kW)', 'z5_AC1(kW)',
           'z5_Light(kW)', 'z5_Plug(kW)' ]

load_degC = ['z1_S1(degC)', 'z2_S1(degC)', 'z4_S1(degC)', 'z5_S1(degC)']

load_rh = ['z1_S1(RH%)', 'z2_S1(RH%)', 'z4_S1(RH%)', 'z5_S1(RH%)' ]

load_lux = ['z1_S1(lux)', 'z2_S1(lux)', 'z4_S1(lux)', 'z5_S1(lux)' ]

data['total_kw'] = data.loc[ :, load_kw].sum(axis = 1)

data['total_lux'] = (data.loc[ :, load_lux].sum(axis=1))/100

data['total_rh'] = (data.loc[ :, load_rh].sum(axis=1))/2

data['total_degC'] = (data.loc[ :, load_degC].sum(axis=1))*2


data.head(5)





,Date,z1_AC1(kW),z1_AC2(kW),z1_AC3(kW),z1_AC4(kW),z1_Light(kW),z1_Plug(kW),z1_S1(degC),z1_S1(RH%),z1_S1(lux),...,z5_AC1(kW),z5_Light(kW),z5_Plug(kW),z5_S1(degC),z5_S1(RH%),z5_S1(lux),total_kw,total_lux,total_rh,total_degC
1,2018-07-01 00:01:00,0.0,0.0,0.0,8.74,0.00,0.23,28.29,62.50,0.0,...,0.0,0.0,0.14,27.73,62.87,0.0,9.74,0.0,95.555,166.66
2,2018-07-01 00:02:00,0.0,0.0,0.0,8.61,0.00,0.22,28.29,62.50,0.0,...,0.0,0.0,0.14,27.72,62.88,0.0,9.61,0.0,95.560,166.64
3,2018-07-01 00:03:00,0.0,0.0,0.0,8.81,0.01,0.23,28.29,62.50,0.0,...,0.0,0.0,0.14,27.72,62.88,0.0,9.83,0.0,95.555,166.64
4,2018-07-01 00:04:00,0.0,0.0,0.0,8.89,0.00,0.22,28.29,62.51,0.0,...,0.0,0.0,0.14,27.72,62.88,0.0,9.88,0.0,95.560,166.62
5,2018-07-01 00:05:00,0.0,0.0,0.0,8.68,0.00,0.23,28.29,62.51,0.0,...,0.0,0.0,0.14,27.72,62.88,0.0,9.69,0.0,95.560,166.62


# Target Variable Dataframe and its Statistical Data

In [66]:
new_data = pd.DataFrame()
new_data = data.loc[:, ['total_kw', 'total_lux', 'total_rh', 'total_degC']]

new_data.head(5)

,total_kw,total_lux,total_rh,total_degC
1,9.74,0.0,95.555,166.66
2,9.61,0.0,95.560,166.64
3,9.83,0.0,95.555,166.64
4,9.88,0.0,95.560,166.62
5,9.69,0.0,95.560,166.62


In [67]:
new_data.describe()

,total_kw,total_lux,total_rh,total_degC
count,790559.000000,790559.000000,790559.000000,790559.000000
mean,20.095084,0.530925,82.484251,126.139832
std,33.410005,0.870776,58.149937,89.779749
min,0.000000,0.000000,0.000000,0.000000
25%,1.080000,0.000000,0.000000,0.000000
50%,2.170000,0.000000,109.505000,160.600000
75%,14.210000,0.920000,129.810000,201.780000
max,234.800000,3.040000,156.750000,245.280000


# Data Preprocessing

### Data Normalization using MinMaxScaler

In [68]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(new_data)

In [70]:
data_scaled

array([[0.04148211, 0.        , 0.60960128, 0.67946836],
       [0.04092845, 0.        , 0.60963317, 0.67938682],
       [0.04186542, 0.        , 0.60960128, 0.67938682],
       ...,
       [0.00413118, 0.        , 0.83177033, 0.88837247],
       [0.00370528, 0.        , 0.83180223, 0.88902479],
       [0.00396082, 0.        , 0.83180223, 0.88902479]])

In [71]:
features = data_scaled
target = data_scaled[:,0]

# Training

In [72]:
TimeseriesGenerator(features, target, length=2, sampling_rate=1, batch_size=1)[0]

(array([[[0.04148211, 0.        , 0.60960128, 0.67946836],
         [0.04092845, 0.        , 0.60963317, 0.67938682]]]),
 array([0.04186542]))

### Split Data into Train and Test (80-20)

In [73]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=123, shuffle = False)

In [74]:
x_train.shape

(632447, 4)

In [75]:
x_test.shape

(158112, 4)

In [88]:
win_length= 720
batch_size = 224
num_features=4

train_generator = TimeseriesGenerator(x_train, y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(x_test, y_test, length=win_length, sampling_rate=1, batch_size=batch_size)


In [77]:
train_generator[0]

(array([[[0.04148211, 0.        , 0.60960128, 0.67946836],
         [0.04092845, 0.        , 0.60963317, 0.67938682],
         [0.04186542, 0.        , 0.60960128, 0.67938682],
         ...,
         [0.05144804, 0.04276316, 0.79100478, 0.93778539],
         [0.04808348, 0.04276316, 0.79097289, 0.93770385],
         [0.04914821, 0.04276316, 0.79097289, 0.93778539]],
 
        [[0.04092845, 0.        , 0.60963317, 0.67938682],
         [0.04186542, 0.        , 0.60960128, 0.67938682],
         [0.04207836, 0.        , 0.60963317, 0.67930528],
         ...,
         [0.04808348, 0.04276316, 0.79097289, 0.93770385],
         [0.04914821, 0.04276316, 0.79097289, 0.93778539],
         [0.0467632 , 0.04276316, 0.79090909, 0.93778539]],
 
        [[0.04186542, 0.        , 0.60960128, 0.67938682],
         [0.04207836, 0.        , 0.60963317, 0.67930528],
         [0.04126917, 0.        , 0.60963317, 0.67930528],
         ...,
         [0.04914821, 0.04276316, 0.79097289, 0.93778539],
        

### Neural Network Layers

In [89]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape = (win_length, num_features), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.LSTM(32, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(16, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1))

In [90]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 720, 32)           4736      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 720, 32)           0         
                                                                 
 lstm_10 (LSTM)              (None, 720, 32)           8320      
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 720, 32)           0         
                                                                 
 dropout_6 (Dropout)         (None, 720, 32)           0         
                                                                 
 lstm_11 (LSTM)              (None, 16)                3136      
                                                                 
 dropout_7 (Dropout)         (None, 16)               

### Early Stopping Criterias

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode=min)

model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])

history = model.fit(train_generator, epochs=50, validation_data=test_generator, shuffle=False, callbacks=[early_stopping])

Epoch 1/50
3291/3291 [==============================] - 246s 73ms/step - loss: 0.0024 - mean_absolute_error: 0.0261 - val_loss: 4.0932e-04 - val_mean_absolute_error: 0.0105
Epoch 2/50
3291/3291 [==============================] - 238s 72ms/step - loss: 0.0011 - mean_absolute_error: 0.0196 - val_loss: 2.3721e-04 - val_mean_absolute_error: 0.0099
Epoch 3/50
3291/3291 [==============================] - 256s 78ms/step - loss: 9.0379e-04 - mean_absolute_error: 0.0182 - val_loss: 1.7380e-04 - val_mean_absolute_error: 0.0078
Epoch 4/50
3131/3291 [===========================>..] - ETA: 10s - loss: 8.5478e-04 - mean_absolute_error: 0.0177

In [ ]:
model.evaluate_generator(test_generator, verbose=0)

In [ ]:
predictions=model.predict_generator(test_generator)

In [ ]:
predictions.shape[0]

In [ ]:
predictions

In [ ]:
y_test

In [ ]:
x_test

In [ ]:
x_test[:,1:][win_length:]

In [ ]:
df_pred= pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[:,1:][win_length:])],axis=1)

### Reverse Transformation of Scaled Data into Original Data

In [ ]:
rev_trans = scaler.inverse_transform(df_pred)

In [ ]:
rev_trans

## Output

In [ ]:
df_final = data[predictions.shape[0]*-1:]

In [ ]:
df_final.count()

In [ ]:
df_final['App_Pred'] = rev_trans[:,0]

In [ ]:
df_final.head(5)

In [ ]:
df_final[['total_kw', 'App_Pred']].iplot()